<a href="https://colab.research.google.com/github/georgesteve/Research-Assitant-ML-ROJ/blob/main/Space_Weather_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
column_names=['Año','Day','Tiempo','Vector_B','BX','BY','BZ','SW_Proton','Kp_index','R','Dst-index','f10.7_index','AE-index']
Data = pd.read_csv('/content/gdrive/My Drive/Data-completa/omni2_sjyVjCUEsu.lst.txt',delim_whitespace=True,names=column_names)


In [ ]:
Data['date_time'] = pd.to_datetime(Data['Año']*1000+Data['Day'], format='%Y%j')+pd.Series([pd.Timedelta(hours=x) for x in Data['Tiempo']])-pd.Timedelta(hours=5)

In [ ]:
df = Data.drop('Año', 1)
df = df.drop('Day', 1)
df = df.drop('Tiempo', 1)
df.set_index('date_time', inplace=True)

In [ ]:
s4 = pd.read_csv('/content/gdrive/My Drive/Data-completa/S4 2015-2017.csv',index_col=False) 
s4=s4.drop(columns=['Unnamed: 0'])

In [ ]:
s4['Az']=s4['Az']+90
s4['Az']=s4['Az'].apply(lambda x: x-360 if x>360 else x)
s4['Az']=s4['Az']*np.pi/(180)
s4['date_time'] = pd.to_datetime(s4['Año']*1000+s4['Day'], format='%y%j')+pd.Series([pd.Timedelta(seconds=x) for x in s4['Tiempo']])-pd.Timedelta(hours=5)


In [ ]:
s4_clean=s4[s4['Elv']>30]
s4_clean=s4_clean[np.isfinite(s4_clean['S4'])]
s4_clean.set_index('date_time', inplace=True)


In [ ]:
print(s4_clean)


                     Año  Day  Tiempo  PRN    S4        Az   Elv
date_time                                                       
2014-12-31 19:00:27   15    1      27   16  0.04  1.591571  66.6
2014-12-31 19:00:27   15    1      27   22  0.03  1.669096  70.4
2014-12-31 19:00:27   15    1      27   27  0.08  1.575091  35.6
2014-12-31 19:00:27   15    1      27  138  0.04  1.603817  50.6
2014-12-31 19:01:27   15    1      87   16  0.03  1.591937  66.8
...                  ...  ...     ...  ...   ...       ...   ...
2017-11-06 15:23:22   17  310   73402  138  0.04  1.603786  50.6
2017-11-06 15:24:22   17  310   73462   21  0.03  1.616915  59.0
2017-11-06 15:24:22   17  310   73462   25  0.07  1.669797  45.4
2017-11-06 15:24:22   17  310   73462   29  0.05  1.679392  37.8
2017-11-06 15:24:22   17  310   73462  138  0.04  1.603786  50.6

[7955542 rows x 7 columns]


In [ ]:
s4_5min=s4_clean.resample("5min").mean()
Data_5min=df.resample("5min").pad()

In [ ]:
s4_5min.head(80)

,Año,Day,Tiempo,PRN,S4,Az,Elv
date_time,,,,,,,
2014-12-31 19:00:00,15.0,1.0,147.000000,50.75000,0.043500,1.610124,55.830000
2014-12-31 19:05:00,15.0,1.0,447.000000,50.75000,0.039500,1.610694,55.860000
2014-12-31 19:10:00,15.0,1.0,752.000000,47.62500,0.048333,1.611870,51.662500
2014-12-31 19:15:00,15.0,1.0,1047.000000,47.00000,0.046000,1.612371,51.092000
2014-12-31 19:20:00,15.0,1.0,1347.000000,41.50000,0.055667,1.617576,47.876667
...,...,...,...,...,...,...,...
2015-01-01 01:15:00,15.0,1.0,22647.000000,24.00000,0.063111,1.623986,46.642222
2015-01-01 01:20:00,15.0,1.0,22947.000000,24.00000,0.061778,1.628228,46.095556
2015-01-01 01:25:00,15.0,1.0,23247.000000,24.00000,0.064667,1.627631,45.544444


In [ ]:
Data_5min.head()

,Vector_B,BX,BY,BZ,SW_Proton,Kp_index,R,Dst-index,f10.7_index,AE-index
date_time,,,,,,,,,,
2014-12-31 19:00:00,6.1,5.1,-2.9,1.1,2.8,23,104,-14,132.9,42
2014-12-31 19:05:00,6.1,5.1,-2.9,1.1,2.8,23,104,-14,132.9,42
2014-12-31 19:10:00,6.1,5.1,-2.9,1.1,2.8,23,104,-14,132.9,42
2014-12-31 19:15:00,6.1,5.1,-2.9,1.1,2.8,23,104,-14,132.9,42
2014-12-31 19:20:00,6.1,5.1,-2.9,1.1,2.8,23,104,-14,132.9,42


In [ ]:
merge=pd.merge(s4_5min,Data_5min, how='inner', left_index=True, right_index=True)

In [ ]:
merge.head()

,Año,Day,Tiempo,PRN,S4,Az,Elv,Vector_B,BX,BY,BZ,SW_Proton,Kp_index,R,Dst-index,f10.7_index,AE-index
date_time,,,,,,,,,,,,,,,,,
2014-12-31 19:00:00,15.0,1.0,147.0,50.750,0.043500,1.610124,55.830000,6.1,5.1,-2.9,1.1,2.8,23,104,-14,132.9,42
2014-12-31 19:05:00,15.0,1.0,447.0,50.750,0.039500,1.610694,55.860000,6.1,5.1,-2.9,1.1,2.8,23,104,-14,132.9,42
2014-12-31 19:10:00,15.0,1.0,752.0,47.625,0.048333,1.611870,51.662500,6.1,5.1,-2.9,1.1,2.8,23,104,-14,132.9,42
2014-12-31 19:15:00,15.0,1.0,1047.0,47.000,0.046000,1.612371,51.092000,6.1,5.1,-2.9,1.1,2.8,23,104,-14,132.9,42
2014-12-31 19:20:00,15.0,1.0,1347.0,41.500,0.055667,1.617576,47.876667,6.1,5.1,-2.9,1.1,2.8,23,104,-14,132.9,42


In [ ]:
from google.colab import files
merge.to_csv('SW-S4-2015-2017.csv')
files.download('SW-S4-2015-2017.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>